# **Fuzzy Logic Tip Recommender**
#### ECE 449 D31 Lab 4
**Ayra Qutub** 1708104


---
This report discusses the design and implementation of a fuzzy inference system (FIS) that recommends a tip percentage based on a multi-dimensional analysis of food quality and service quality. The project builds on the "Tipper" example in the skfuzzy library and extends it by implementing a fuzzy tree structure, allowing the program to evaluate individual attributes that collectively define food and service quality. These are then used to recommend a tip amount.




In [ ]:
!pip install scikit-fuzzy
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 8.8 MB/s eta 0:00:00


## Determining Food Quality
This function evaluates the quality of the food based on three inputs: temperature, flavor, and portion size. These are measured on a scale from 0 to 10, each representing how favorable the user determined these aspects to be, from poor to good.

The membership functions are mathematical functions that define how each point in the input space is mapped to a membership value (degree of membership) between 0 and 1. This was done using `trimf()`, which maps to a triangualar shape with specified dimensions.
- poor: 0 to 4, peak at 0
- average: 2 to 8, peak at 5
- good: 6 to 10, peak at 10

The membership functions were created with `trimf()` instead of `automf()` because the latter tends to skew values towards the average. When testing using `automf()`, tips calculated would be near the middle (12.5%), even when inputting extreme value ratings (e.g all 0s or all 10s). Setting custom ranges helps to counteract this and results in tip recommendations that are more reflective of the user's opinions.

Five fuzzy rules are defined to map combinations of these input qualities to an output food quality.
- Rule 1: If any attribute is rated as poor, then the food quality is poor.
- Rule 2: If all attributes are average, the food quality is average.
- Rule 3: If all attributes are good, the food quality is good.
- Rule 4: If any two attributes are good, the food quality is good
- Rule 5: If any attribute is average and no attributes are poor, the food quality is average

After setting the input values, the `compute()` function calculates a defuzzified score for food quality. This scalar value represents the overall food quality based on user input, serving as an input for the next FIS stage.

In [ ]:
def calc_food_quality(temperature_input, flavor_input, portion_size_input):
  ## define variables
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
  food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')

  ## membership functions
  temperature['poor'] = fuzz.trimf(temperature.universe, [0, 0, 4])
  temperature['average'] = fuzz.trimf(temperature.universe, [2, 5, 8])
  temperature['good'] = fuzz.trimf(temperature.universe, [6, 10, 10])

  flavor['poor'] = fuzz.trimf(flavor.universe, [0, 0, 4])
  flavor['average'] = fuzz.trimf(flavor.universe, [2, 5, 8])
  flavor['good'] = fuzz.trimf(flavor.universe, [6, 10, 10])

  portion_size['poor'] = fuzz.trimf(portion_size.universe, [0, 0, 4])
  portion_size['average'] = fuzz.trimf(portion_size.universe, [2, 5, 8])
  portion_size['good'] = fuzz.trimf(portion_size.universe, [6, 10, 10])

  food_quality['poor'] = fuzz.trimf(food_quality.universe, [0, 0, 4])
  food_quality['average'] = fuzz.trimf(food_quality.universe, [2, 5, 8])
  food_quality['good'] = fuzz.trimf(food_quality.universe, [6, 10, 10])

  ## fuzzy rules
  rule1_food = ctrl.Rule(temperature['poor'] | flavor['poor'] | portion_size['poor'], food_quality['poor'])
  rule2_food = ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['average'], food_quality['average'])
  rule3_food = ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality['good'])
  rule4_food = ctrl.Rule((temperature['good'] & flavor['good']) | (flavor['good'] & portion_size['good']) | (temperature['good'] & portion_size['good']), food_quality['good'])
  rule5_food = ctrl.Rule((temperature['average'] | flavor['average'] | portion_size['average']) & ~(temperature['poor'] | flavor['poor'] | portion_size['poor']), food_quality['average'])

  ## create controller
  food_quality_ctrl = ctrl.ControlSystem([rule1_food, rule2_food, rule3_food, rule4_food, rule5_food])
  food_sim = ctrl.ControlSystemSimulation(food_quality_ctrl)
  food_sim.input['temperature'] = temperature_input
  food_sim.input['flavor'] = flavor_input
  food_sim.input['portion_size'] = portion_size_input

  food_sim.compute()
  # print(food_sim.output['food_quality'])
  # food_quality.view(sim=food_sim)

  return food_sim.output['food_quality']

## Determining Service Quality
This function calculates service quality based on attentiveness, friendliness, and speed, again using a scale from 0 to 10.

The membership functions are defined in the same way as above. Five fuzzy rules determine the service quality output:
- Rule 1: If any attribute is poor, the service quality is poor
- Rule 2: If all attributes are average, the service quality is average
- Rule 3: If all attributes are good, the service quality is good.
- Rule 4: If any two attributes are good, the service quality is good
- Rule 5: If any attribute is average and no attributes are poor, the service quality is average

The function then computes a defuzzified service quality score, which is passed to the final `calc_tip` function as an indicator of service quality.

In [ ]:
def calc_service_quality(attentiveness_input, friendliness_input, speed_input):
  ## define variables
  attentiveness= ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
  speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
  service_quality = ctrl.Consequent(np.linspace(0, 10, 11),'service_quality')

  attentiveness['poor'] = fuzz.trimf(attentiveness.universe, [0, 0, 4])
  attentiveness['average'] = fuzz.trimf(attentiveness.universe, [2, 5, 8])
  attentiveness['good'] = fuzz.trimf(attentiveness.universe, [6, 10, 10])

  friendliness['poor'] = fuzz.trimf(friendliness.universe, [0, 0, 4])
  friendliness['average'] = fuzz.trimf(friendliness.universe, [2, 5, 8])
  friendliness['good'] = fuzz.trimf(friendliness.universe, [6, 10, 10])

  speed['poor'] = fuzz.trimf(speed.universe, [0, 0, 4])
  speed['average'] = fuzz.trimf(speed.universe, [2, 5, 8])
  speed['good'] = fuzz.trimf(speed.universe, [6, 10, 10])

  service_quality['poor'] = fuzz.trimf(service_quality.universe, [0, 0, 4])
  service_quality['average'] = fuzz.trimf(service_quality.universe, [2, 5, 8])
  service_quality['good'] = fuzz.trimf(service_quality.universe, [6, 10, 10])

  ## fuzzy rules
  rule1_service = ctrl.Rule(attentiveness['poor'] | friendliness['poor'] | speed['poor'], service_quality['poor'])
  rule2_service = ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed['average'], service_quality['average'])
  rule3_service = ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['good'], service_quality['good'])
  rule4_service = ctrl.Rule((attentiveness['good'] & friendliness['good']) | (friendliness['good'] & speed['good']) | (attentiveness['good'] & speed['good']), service_quality['good'])
  rule5_service = ctrl.Rule((attentiveness['average'] | friendliness['average'] | speed['average']) & ~(attentiveness['poor'] | friendliness['poor'] | speed['poor']), service_quality['average'])

  service_quality_ctrl = ctrl.ControlSystem([rule1_service, rule2_service, rule3_service, rule4_service, rule5_service])
  service_sim = ctrl.ControlSystemSimulation(service_quality_ctrl)
  service_sim.input['attentiveness'] = attentiveness_input
  service_sim.input['friendliness'] = friendliness_input
  service_sim.input['speed'] = speed_input

  service_sim.compute()
  # print(service_sim.output['service_quality'])
  # service_quality.view(sim=service_sim)

  return service_sim.output['service_quality']

## Determining Tip Amount
This function computes the recommended tip percentage based on the calculated values of food and service quality.

 The function defines antecedents for food and service quality, each with three fuzzy sets (poor, average, and good). The consequent, *tip*, ranges from 0% to 25%, allowing for a graded scale of tips. The membership functions are defined in the same way as explained earlier.

 Six fuzzy rules govern the tip calculation:
 - Rule 1: If both qualities are poor, the tip is low
 - Rule 2: If both qualities are average, the tip is medium
 - Rule 3: If both qualities are good, the tip is high
 - Rule 4: If one quality is good and the other is average, the tip is high
 - Rule 5: If one quality is good and the other is poor, the tip is medium
 - Rule 6: If one quality is average and the other is poor, the tip is low

Using the inputs from the previous two functions, the defuzzified tip percentage is calculated, providing a recommendation based on the quality metrics.


In [ ]:
def calc_tip(food_quality_output, service_quality_output):
  food = ctrl.Antecedent(np.linspace(0, 10, 11), 'food')
  service = ctrl.Antecedent(np.linspace(0, 10, 11), 'service')
  tip = ctrl.Consequent(np.linspace(0, 25, 26), 'tip')

  food['poor'] = fuzz.trimf(food.universe, [0, 0, 4])
  food['average'] = fuzz.trimf(food.universe, [2, 5, 8])
  food['good'] = fuzz.trimf(food.universe, [6, 10, 10])

  service['poor'] = fuzz.trimf(service.universe, [0, 0, 4])
  service['average'] = fuzz.trimf(service.universe, [2, 5, 8])
  service['good'] = fuzz.trimf(service.universe, [6, 10, 10])

  tip['low'] = fuzz.trimf(tip.universe, [0, 0, 10])
  tip['medium'] = fuzz.trimf(tip.universe, [8, 12.5, 17])
  tip['high'] = fuzz.trimf(tip.universe, [16, 25, 25])

  rule1_tip = ctrl.Rule(food['poor'] & service['poor'], tip['low'])
  rule2_tip = ctrl.Rule(food['average'] & service['average'], tip['medium'])
  rule3_tip = ctrl.Rule(food['good'] & service['good'], tip['high'])
  rule4_tip = ctrl.Rule((food['good'] & service['average']) | (food['average'] & service['good']), tip['high'])
  rule5_tip = ctrl.Rule((food['good'] & service['poor']) | (food['poor'] & service['good']), tip['medium'])
  rule6_tip = ctrl.Rule((food['average'] & service['poor']) | (food['poor'] & service['average']), tip['low'])

  tip_ctrl = ctrl.ControlSystem([rule1_tip, rule2_tip, rule3_tip, rule4_tip, rule5_tip, rule6_tip])
  tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)
  tip_sim.input['food'] = food_quality_output
  tip_sim.input['service'] = service_quality_output

  tip_sim.compute()
  # print(tip_sim.output['tip'])
  # tip.view(sim=tip_sim)
  return tip_sim.output['tip']

## Validating Inputs
This function validates that the user input for each quality dimension is a numerical value between 0 and 10. This is critical for ensuring that all inputs to the FIS are within an acceptable range, as values outside of this range would lead to inaccurate or unexpected FIS behavior.

In [ ]:
def is_valid_input(value):
    try:
        value = float(value)
        if 0 <= value <= 10:
            return value
        else:
            print("Please enter a number between 0 and 10.")
            return None
    except ValueError:
        print("Invalid input. Please enter a valid number.")
        return None

## Tip Recommender
The `main` function is the pipeline which integrates all other functions, prompting the user to rate food and service aspects on a 0-10 scale and validating each input through is_valid_input.

The user is prompted to input ratings for temperature, flavor, portion size (food quality factors), and attentiveness, friendliness, speed (service quality factors). These inputs are passed through `calc_food_quality` and `calc_service_quality`, and the outputs are used by `calc_tip` to compute the recommended tip percentage.

After each calculation, the user is asked if they wish to continue, facilitating multiple rounds of tip recommendations if desired.

In [ ]:
if __name__ == '__main__':
  print("Welcome to the tip calculator!")
  while True:
    print("Please rate the following items on a scale of 0-10:")
    temperature_input = flavor_input = portion_size_input = None
    attentiveness_input = friendliness_input = speed_input = None
    while temperature_input == None:
      temperature_input = is_valid_input(input("Food - Temperature: "))
    while flavor_input == None:
      flavor_input = is_valid_input(input("Food - Flavor: "))
    while portion_size_input == None:
      portion_size_input = is_valid_input(input("Food - Portion Size: "))
    while attentiveness_input == None:
      attentiveness_input = is_valid_input(input("Service - Attentiveness: "))
    while friendliness_input == None:
      friendliness_input = is_valid_input(input("Service - Friendliness: "))
    while speed_input == None:
      speed_input = is_valid_input(input("Service - Speed: "))

    food_quality_output = calc_food_quality(temperature_input, flavor_input, portion_size_input)
    service_quality_output = calc_service_quality(attentiveness_input, friendliness_input, speed_input)
    tip_output = calc_tip(food_quality_output, service_quality_output)
    print(f"Your tip should be: {tip_output:.1f}%")

    if input("Would you like to calculate another tip? Enter 'yes' to continue: ") != "yes":
      print("Thank you for using the tip calculator!")
      break

Welcome to the tip calculator!
Please rate the following items on a scale of 0-10:
Food - Temperature: 10
Food - Flavor: 10
Food - Portion Size: 10
Service - Attentiveness: 10
Service - Friendliness: 10
Service - Speed: 10
Your tip should be: 21.8%
Would you like to calculate another tip? Enter 'yes' to continue: 
Thank you for using the tip calculator!


## Conclusion
By breaking down "quality" into more measurable attributes, this FIS leverages each dimension's fuzzy logic to address the "curse of dimensionality" in multi-dimensional rule-based systems.

The fuzzy tree structure effectively reduces complexity by combining inputs through intermediate FIS stages, which ensures that the final recommendation is both interpretable and computationally feasible. The chosen architecture resembles a hierarchical fuzzy system, where food and service quality evaluations lead to a final tip recommendation. This modular approach simplifies rule management, as only a subset of all possible rules is needed at each FIS level.